# This notebook cover fundamental concepts of tensors using Tensorflow.

More specificly, we're going to cover:
* Introduction to tensors
* Getting informations form tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with Tensorflow (or TPUs)


## Introduction to Tensors

In [2]:
# Import TnsorFLow
import tensorflow as tf
print(tf.__version__)

2.6.0


### Creating tensors with tf.constant

In [3]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [6]:
# Check the dimension of our vector
vector.ndim

1

In [7]:
# Create a matrix
matrix = tf.constant([[1,2,3],
                      [4,5,6],
                      [7,8,9]
                      ])
matrix

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)>

In [8]:
# Check the dimension of the matrix
matrix.ndim

2

In [9]:
# Create another matrix
another_matrix = tf.constant([[10.,7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify data type
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [10]:
# Let's create a tensor
tensor = tf.constant([
                      [[1, 2 , 3],
                       [4, 5 , 6]],
                      [[7, 8 , 9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]
                     ])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

Covered so far:
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: n-dimensional array of numbers 

### Creating tensors with tf.variable 

In [11]:
# Create the same tensor as above with tf.variable
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

In [12]:
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>

In [13]:
unchangeable_tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>

In [14]:
# Change changeable tensor
changeable_tensor[0] = 123

TypeError: ignored

In [15]:
# Try the same with .assign()
changeable_tensor[0].assign(123)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([123,   7], dtype=int32)>

In [17]:
# try change unchangeable tensor
unchangeable_tensor[0].assign(123) 

AttributeError: ignored

tf.constant() create immutable tensor

### Creating ranodm tensors

Random tensors are tensors of some abritrary size which contain random numbers.

In [18]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42).normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [19]:
random_2 = tf.random.Generator.from_seed(42).normal(shape=(3,2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [20]:
# Are random_1 and random_2 equal?
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

In [21]:
### Shuffle the order of elements in a tensor
# Shuffle a tensor (valuable for when you want to shiffle data so the inherent order doesn't effect learning)

not_shuffled = tf.constant([[1,2],
                            [3,4],
                            [5,6],
                            [7,8],
                            [9,10]])
not_shuffled

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10]], dtype=int32)>

In [22]:
tf.random.set_seed(42) # global level random seed
shuffled = tf.random.shuffle(not_shuffled, seed=42) # operation level random seed
shuffled

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 9, 10]], dtype=int32)>

### Other ways to make an tensor


In [23]:
# Create a tensor full of ones
tf.ones(shape=(3,3), dtype=tf.int8)

<tf.Tensor: shape=(3, 3), dtype=int8, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int8)>

In [24]:
# Create a tensor full of zeros
tf.zeros(shape=(3,3), dtype=tf.int8)

<tf.Tensor: shape=(3, 3), dtype=int8, numpy=
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tesnor is that tensor can be run on a GPU for faster computing.

In [25]:
# Create NumPy array
import numpy as np
np_A = np.arange(1, 25, dtype=np.int32)
np_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [26]:
A = tf.constant(np_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [27]:
B = tf.constant(np_A, shape=(4,6))
B

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12],
       [13, 14, 15, 16, 17, 18],
       [19, 20, 21, 22, 23, 24]], dtype=int32)>

In [28]:
B.ndim

2

### Getting informations from tensors

When dealing with tensors you probably want to be aware of the following
* Shape - The length (number of elements) of each of the dimensions of a tensor - `tensor.shape`
* Rank - number of tensors dimensions - `tensor.ndim`
* Axis or dimension - A particular dimension of a tensor - `tensor[0]`, `tensor[:,1]`
* Size - The total number of items in the tensor - `tf.size(tensor)`

In [29]:
# Create a rank 4 tensor
rank_4_tsr = tf.zeros(shape=(2,3,3,2), dtype=tf.int8)
rank_4_tsr

<tf.Tensor: shape=(2, 3, 3, 2), dtype=int8, numpy=
array([[[[0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0]]]], dtype=int8)>

In [30]:
rank_4_tsr[0]

<tf.Tensor: shape=(3, 3, 2), dtype=int8, numpy=
array([[[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]]], dtype=int8)>

In [31]:
rank_4_tsr[0][0]

<tf.Tensor: shape=(3, 2), dtype=int8, numpy=
array([[0, 0],
       [0, 0],
       [0, 0]], dtype=int8)>

In [32]:
rank_4_tsr[0][0][0]

<tf.Tensor: shape=(2,), dtype=int8, numpy=array([0, 0], dtype=int8)>

In [33]:
rank_4_tsr.shape, rank_4_tsr.ndim, tf.size(rank_4_tsr).numpy()

(TensorShape([2, 3, 3, 2]), 4, 36)

### Indexing tensors

In [34]:
# Get the first 2 elements of each dimension
rank_4_tsr[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int8, numpy=
array([[[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]]], dtype=int8)>

In [35]:
# Get the first element from each dimension from each index except for the final one
rank_4_tsr[:1,:1,:1]

<tf.Tensor: shape=(1, 1, 1, 2), dtype=int8, numpy=array([[[[0, 0]]]], dtype=int8)>

In [36]:
# Create a rank 2 tensor
rank_2_tsr = tf.Variable([[12,34],
                             [56,78]], dtype=tf.int8)
rank_2_tsr

<tf.Variable 'Variable:0' shape=(2, 2) dtype=int8, numpy=
array([[12, 34],
       [56, 78]], dtype=int8)>

In [37]:
# Get the last item of each row of 2 rank tensor
rank_2_tsr[:,-1]

<tf.Tensor: shape=(2,), dtype=int8, numpy=array([34, 78], dtype=int8)>

In [38]:
# Add in extra dimension to rank 2 tensor
rank_3_tsr = rank_2_tsr[..., tf.newaxis]
rank_3_tsr

<tf.Tensor: shape=(2, 2, 1), dtype=int8, numpy=
array([[[12],
        [34]],

       [[56],
        [78]]], dtype=int8)>

In [39]:
# Alternative to above
rank_3_tsr_alt = tf.expand_dims(rank_2_tsr, axis=-1) # "-1" means expand the final axis
rank_3_tsr_alt

<tf.Tensor: shape=(2, 2, 1), dtype=int8, numpy=
array([[[12],
        [34]],

       [[56],
        [78]]], dtype=int8)>

In [40]:
tf.expand_dims(rank_2_tsr, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int8, numpy=
array([[[12, 34],
        [56, 78]]], dtype=int8)>

In [41]:
tf.expand_dims(rank_2_tsr, axis=1)

<tf.Tensor: shape=(2, 1, 2), dtype=int8, numpy=
array([[[12, 34]],

       [[56, 78]]], dtype=int8)>

### Manipulating tensors and operations on a tensors

In [42]:
# Adding values
tensor + 10

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[11, 12, 13],
        [14, 15, 16]],

       [[17, 18, 19],
        [20, 21, 22]],

       [[23, 24, 25],
        [26, 27, 28]]], dtype=int32)>

In [43]:
tensor # Original tensor will remain unchanged

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [44]:
# Multiplication
tensor * 123

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 123,  246,  369],
        [ 492,  615,  738]],

       [[ 861,  984, 1107],
        [1230, 1353, 1476]],

       [[1599, 1722, 1845],
        [1968, 2091, 2214]]], dtype=int32)>

In [45]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[-9, -8, -7],
        [-6, -5, -4]],

       [[-3, -2, -1],
        [ 0,  1,  2]],

       [[ 3,  4,  5],
        [ 6,  7,  8]]], dtype=int32)>

Other mathematical operations are also available.

Tensor oparations also have equivalent functions, those can be used to speed up calculation by using GPU/TPU 


In [46]:
tf.math.multiply(tensor, 123)

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 123,  246,  369],
        [ 492,  615,  738]],

       [[ 861,  984, 1107],
        [1230, 1353, 1476]],

       [[1599, 1722, 1845],
        [1968, 2091, 2214]]], dtype=int32)>

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operation.

In [47]:
# Matrix multiplication in tensorflow
print(matrix)
tf.matmul(matrix, matrix)

tf.Tensor(
[[1 2 3]
 [4 5 6]
 [7 8 9]], shape=(3, 3), dtype=int32)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 30,  36,  42],
       [ 66,  81,  96],
       [102, 126, 150]], dtype=int32)>

In [48]:
# Matrix multiplication with Python operator "@"
matrix @ matrix

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 30,  36,  42],
       [ 66,  81,  96],
       [102, 126, 150]], dtype=int32)>

In [49]:
# Create a (3, 2) tensor
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
X

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [50]:
# Create anoteher (3, 2) tensor
Y = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [51]:
X.shape, Y.shape

(TensorShape([3, 2]), TensorShape([3, 2]))

In [52]:
# Try to matrix multiply tensors of the same shape
X @ Y

InvalidArgumentError: ignored

To be able multiply matrices, inner dimensions must match. Product will have a shape of outer dimensions.

In [ ]:
# We can transponse one of the matricies.
Y_t = tf.transpose(Y)
Y_t

In [ ]:
X @ Y_t

In [ ]:
tf.reshape(Y, shape=(2,3))

In [ ]:
# The similar result can be achived using tf.reshape()
X @ tf.reshape(Y, shape=(2,3))

**The dot Product**

Matrix multiplication is also reffered to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [ ]:
# Perdorm the dot product on X and Y (X, or Y must be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

### Changing the data type of the tensor

Nice resource about mixed precision oparations on GPU:
https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html

In [ ]:
# Create tensor with default data dtype
B = tf.constant([1.7, 7.4])
B.dtype

In [ ]:
C = tf.constant([1, 7])
C.dtype

In [ ]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, tf.float16)
D, D.dtype

In [ ]:
# Change int32 to float32
E = tf.cast(C, dtype=tf.int32)
E, E.dtype

### Aggregating tensor

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Get the absolute values
A = tf.constant([-1, -2])
tf.abs(A)

Selected forms of aggreagtion:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor
* Variance
* Standard deviation

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
B = tf.constant(np.random.randint(0, 100, size=50))
B

In [ ]:
# Find the minimum
tf.reduce_min(B)

In [ ]:
# Find the maximum
tf.reduce_max(B)

In [ ]:
# Find the mean
tf.reduce_mean(B)

In [ ]:
# Find the sum
tf.reduce_sum(B)

In [ ]:
# Find the Variance
B = tf.cast(B, dtype=tf.complex64)
tf.math.reduce_variance(B)

In [ ]:
# Find the Standard deviation
tf.math.reduce_std(B)

### Positional maximum and minimum

In [ ]:
# Index of min element
B = tf.cast(B, dtype=tf.float32)
tf.argmin(B)

In [ ]:
# Index of max element
tf.argmax(B)

### Squeezing a tensor (removing all single dimensions)

In [ ]:
A = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
A

In [ ]:
tf.squeeze(A)

### One-hot encoded tensors

In [ ]:
# Create a list
some_list = [0, 1, 2]

# One hot encode list create above
depth = len(some_list)
tf.one_hot(some_list, depth=depth)

In [ ]:
# specify custom values for one hot encoding
tf.one_hot(some_list, depth=depth, on_value="idk", off_value="idk_either")

### More math operations on Tensors

Operations:
* Squaring
* log
* sqrt

In [ ]:
# Create example tensor
A = tf.range(1,10)

In [ ]:
# Squaring 
tf.square(A)

In [ ]:
# Square root (error with non-float type)
A = tf.cast(A, dtype=tf.float32)
tf.sqrt(A)

In [ ]:
# log (error with non-float type)
tf.math.log(A)

### Tensors and NumPy

TensorFLow works great with NumPy arrays.

In [ ]:
# Tensor from NumPy array
A = tf.constant(np.array([3., 7., 1.]))
A

In [ ]:
# Convert our tensor back to NumPy array
np.array(A)

In [ ]:
A.numpy()

In [ ]:
# Tensors and NumPy arrays have slightly different default types
numpy_B = np.array([3., 7., 1.])
tensor_B = tf.constant([3., 7., 1.])
numpy_B.dtype, tensor_B.dtype

### Finding access to GPUs

In [2]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
!nvidia-smi

Wed Sep  8 16:00:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------